In [1]:
cd langdata

/projects/metis2/anubha/research/Multilingual-Fig-QA/langdata


In [2]:
#Reading Dataset

In [3]:
ls

en_dev.csv  en_train.csv  hi.csv  id.csv  jv.csv  kn.csv  su.csv  sw.csv


In [4]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(42)

### Experiment 6.2.3

In [5]:
import pandas as pd

In [6]:
# load english data
df_original = pd.read_csv("en_train.csv")
df_original.head()

,startphrase,ending1,ending2,labels
0,Her word had the strength of titanium.,Her promises can be believed.,Her promises cannot be trusted.,0
1,Her word had the strength of a wine glass.,Her promises can be believed.,Her promises cannot be trusted.,1
2,His kisses have the passion of lovers meeting ...,His kisses are demonstrative and intense.,His kiss is unemotional.,0
3,His kisses have the passion of a couple in a l...,His kisses are demonstrative and intense.,His kiss is unemotional.,1
4,This winter is as cold as my mother-in-law tow...,It's very cold,It's pretty warm,0


In [283]:
#load other data
df = pd.read_csv("sw.csv")
df = df.reindex(df_original.columns, axis=1)
df.head()

,startphrase,ending1,ending2,labels
0,Watoto ni malaika,Watoto daima hawana hatia,Yeye daima ni mwenye dhambi,0
1,Yeye ni mnyama.,Watoto daima hawana hatia,Yeye daima ni mwenye dhambi,1
2,Maneno yake ni kama kisu,Maneno yako yanaumiza,Maneno yake yanafariji,0
3,Maneno yake ni kama kinanda,Maneno yako yanaumiza,Maneno yake yanafariji,1
4,Tabasamu lake huangaza chumba kizima,Ana tabasamu zuri,Tabasamu lake linatisha,0


In [284]:
#Add id to the loaded data
count =0
c=[]
for i in range(int(df.shape[0]/2)):
    c.append(count)
    c.append(count)
    count+=1
df["id"] = c  

In [285]:
df.head()

,startphrase,ending1,ending2,labels,id
0,Watoto ni malaika,Watoto daima hawana hatia,Yeye daima ni mwenye dhambi,0,0
1,Yeye ni mnyama.,Watoto daima hawana hatia,Yeye daima ni mwenye dhambi,1,0
2,Maneno yake ni kama kisu,Maneno yako yanaumiza,Maneno yake yanafariji,0,1
3,Maneno yake ni kama kinanda,Maneno yako yanaumiza,Maneno yake yanafariji,1,1
4,Tabasamu lake huangaza chumba kizima,Ana tabasamu zuri,Tabasamu lake linatisha,0,2


In [286]:
# #df_original = english data
# #df = language data
# #k = number of datapoints to be added
# #pair = in pair / not

# def adding_datapoints(df_original, df, k, pair):
#     df_final = pd.DataFrame(columns=df_original.columns)
#     if pair:
#         for i in range(k):
#             p = df.sample(n=1, random_state=42)
#         df_pair = df.sample(n=k, random_state=42)
#         print("number in df_pair", df_pair.shape)
#         for index, r in df_pair.iterrows():
#             r_id = r['id']
#             w = df.loc[df['id'] == r_id]
#             while(w.shape[0] == 0):
#                 p = df.sample(n=1, random_state=42)
#             w.sort_values(by=['labels'], inplace=True)
#             df=df.drop(w.index)
#             w= w.drop(['id'], axis=1)
#             if w.shape[0] < 2:
#                 print("w", r_id)
#             df_final = df_final.append(w)
#     else:
#         df_pair = df.sample(n=k, random_state=42)
#         df=df.drop(df_pair.index)
#         w= df_pair.drop(['id'], axis=1)
#         df_final = df_final.append(w)

# #     df_total = pd.concat([df_original,df_final])
#     df_total = df_final
#     df_test = df.drop(['id'], axis=1)
#     return df_total, df_test
# # 
    

In [287]:
#df_original = english data
#df = language data
#k = number of datapoints to be added
#pair = in pair / not

def adding_datapoints(df_original, df, k, pair):
    df_final = pd.DataFrame(columns=df_original.columns)
    if pair:
        for i in range(k):
            df_pair = df.sample(n=1, random_state=42)
#         df_pair = df.sample(n=k, random_state=42)
#         print("number in df_pair", df_pair.shape)
            for index, r in df_pair.iterrows():
                r_id = r['id']
                w = df.loc[df['id'] == r_id]
                w.sort_values(by=['labels'], inplace=True)
                df=df.drop(w.index)
                w= w.drop(['id'], axis=1)
                df_final = df_final.append(w)
    else:
        for i in range(k):
            df_pair = df.sample(n=1, random_state=42)
#         df_pair = df.sample(n=k, random_state=42)
            df=df.drop(df_pair.index)
            w= df_pair.drop(['id'], axis=1)
            df_final = df_final.append(w)

#     df_total = pd.concat([df_original,df_final])
    df_total = df_final
    df_test = df.drop(['id'], axis=1)
    return df_total, df_test

    

In [300]:
num_samples = 50
lang = "sw"

In [301]:
# #final dataframe, rows added in the end
# df_total = adding_datapoints(df_original, df, 10, True)
# df_total.tail()

#final dataframe, rows added in the end
df_total, df_test = adding_datapoints(df_original, df, num_samples, True)
print(df_total.shape)
df_total

(100, 4)


/tmp/ipykernel_10393/390302092.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  w.sort_values(by=['labels'], inplace=True)
/tmp/ipykernel_10393/390302092.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(w)
/tmp/ipykernel_10393/390302092.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  w.sort_values(by=['labels'], inplace=True)
/tmp/ipykernel_10393/390302092.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead

,startphrase,ending1,ending2,labels
44,Mwendo wake ni mithili ya twiga,Mwendo wake ni wa maringo,Mwendo wake si wa maringo,0
45,Mwendo wake ni wa bata,Mwendo wake ni wa maringo,Mwendo wake si wa maringo,1
46,Wema ni hazina,Ukitendea watu mema yatakurudia,Ukitendea watu mema hayatakurudia,0
47,Wema sio kitenesi,Ukitendea watu mema yatakurudia,Ukitendea watu mema hayatakurudia,1
487,msanii yule ni nyota,amebobea kwenye usanii,hajabobea sana,0
...,...,...,...,...
1011,Wafisadi serekalini wanabebeshwa mbio kama kre...,Wafisadi serekalini wataharakishwa na polisi,Wafisadi serekalini hawanyiwi kitu kamwe,1
573,Mwanasoka yule ni nyota.,Yule amekamilika.,Yule sio kamili.,0
572,Yule ni sumu.,Nampenda sana.,Namchukia sana.,1
579,Yeye ni filimbi.,Yeye ni safi.,Yeye ni chafu.,0


In [302]:
print(df.shape)
print(df.shape)
print(df_test.shape)

(1098, 5)
(1098, 5)
(998, 4)


In [303]:
#loaction : /projects/metis2/anubha/research/Multilingual-Fig-QA/data/addition_data_winogrand -> for paired data addition
#loaction : /projects/metis2/anubha/research/Multilingual-Fig-QA/data/addition_data -> for single data addition
winogrand = r'/projects/metis2/anubha/research/Multilingual-Fig-QA/data/sample_addition_data_winogrand/'

no_winogrand = r'/projects/metis2/anubha/research/Multilingual-Fig-QA/data/sample_addition_data/'



folder_name = lang+ "/"
name = lang + "_" + str(num_samples)
df_total.to_csv(winogrand+'train/' +folder_name+ name+".csv", index = False)
df_test.to_csv(winogrand+'test/'+folder_name+ name+ ".csv", index = False)

### Removing Rows from testing data


### Experiment 6.2.2

In [53]:
import pandas as pd
# load english data
df_original = pd.read_csv("en_train.csv")
df_original.head()

,startphrase,ending1,ending2,labels
0,Her word had the strength of titanium.,Her promises can be believed.,Her promises cannot be trusted.,0
1,Her word had the strength of a wine glass.,Her promises can be believed.,Her promises cannot be trusted.,1
2,His kisses have the passion of lovers meeting ...,His kisses are demonstrative and intense.,His kiss is unemotional.,0
3,His kisses have the passion of a couple in a l...,His kisses are demonstrative and intense.,His kiss is unemotional.,1
4,This winter is as cold as my mother-in-law tow...,It's very cold,It's pretty warm,0


In [54]:
#load other data
df = pd.read_csv("kn.csv")
df = df.reindex(df_original.columns, axis=1)
df.head()

,startphrase,ending1,ending2,labels
0,ಆ ಸ್ಥಳವು ಆಸ್ಪತ್ರೆ ರೀತಿ ಶುಚಿಯಾಗಿದೆ.,ಆ ಸ್ಥಳವು ಶುಚಿಯಾಗಿದೆ,ಆ ಸ್ಥಳವು ಶುಚಿಯಾಗಿಲ್ಲ,0
1,ಆ ಸ್ಥಳವು ಮೀನಿನ ಮಾರುಕಟ್ಟೆ ರೀತಿ ಶುಚಿಯಾಗಿದೆ.,ಆ ಸ್ಥಳವು ಶುಚಿಯಾಗಿದೆ,ಆ ಸ್ಥಳವು ಶುಚಿಯಾಗಿಲ್ಲ,1
2,ಆ ಪ್ರದೇಶ ಯುದ್ಧ ಭೂಮಿಯಷ್ಟು ಹಿಂಸೆಯಿಂದ ಕೂಡಿದೆ.,ಆ ಪ್ರದೇಶ ಹಿಂಸೆಯಿಂದ ಕೂಡಿದೆ,ಆ ಪ್ರದೇಶ ಹಿಂಸೆಯಿಂದ ಕೂಡಿಲ್ಲ,0
3,ಆ ಪ್ರದೇಶ ದೇವಸ್ಥಾನದಷ್ಟು ಹಿಂಸೆಯಿಂದ ಕೂಡಿದೆ.,ಆ ಪ್ರದೇಶ ಹಿಂಸೆಯಿಂದ ಕೂಡಿದೆ,ಆ ಪ್ರದೇಶ ಹಿಂಸೆಯಿಂದ ಕೂಡಿಲ್ಲ,1
4,ಅವರ ಬಾಂಧವ್ಯ ಸ್ನೇಹಿತರ ಹಾಗೆ ಗಟ್ಟಿಯಿದೆ.,ಅವರ ಬಾಂಧವ್ಯ ಗಟ್ಟಿ ಇದೆ,ಅವರ ಬಾಂಧವ್ಯ ಗಟ್ಟಿ ಇಲ್ಲ,0


In [55]:
 df_total = pd.concat([df_original,df])

In [56]:
df_total.head()

,startphrase,ending1,ending2,labels
0,Her word had the strength of titanium.,Her promises can be believed.,Her promises cannot be trusted.,0
1,Her word had the strength of a wine glass.,Her promises can be believed.,Her promises cannot be trusted.,1
2,His kisses have the passion of lovers meeting ...,His kisses are demonstrative and intense.,His kiss is unemotional.,0
3,His kisses have the passion of a couple in a l...,His kisses are demonstrative and intense.,His kiss is unemotional.,1
4,This winter is as cold as my mother-in-law tow...,It's very cold,It's pretty warm,0


In [57]:
print(df_original.shape)
print(df.shape)
print(df_total.shape)

(1458, 4)
(1198, 4)
(2656, 4)


In [58]:
file = r'/projects/metis2/anubha/research/Multilingual-Fig-QA/data/exp6.2.2/'
name = "eng_kn"
df_total.to_csv(file+name+ ".csv", index = False)